In [1]:
import faiss
from langchain_ollama import ChatOllama,OllamaEmbeddings
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
import faiss
import pyttsx3
from langchain_core.output_parsers import PydanticOutputParser,StrOutputParser
from typing import List
import re
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import numpy as np
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_core.prompts import PromptTemplate
import spacy
import pickle
import requests
from scispacy.linking import EntityLinker
import scispacy
import re
from langchain.messages import SystemMessage,HumanMessage,AIMessage,ToolMessage
from langchain.tools import tool
from langchain_community.docstore.in_memory import InMemoryDocstore
import pyobo
import time
from collections import defaultdict


USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/bionik/miniconda3/envs/LLM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#linker = pyobo.get_scispacy_entity_linker("uniprot", filter_for_definitions=False, resolve_abbreviations=True)
nlp = spacy.load('en_ner_jnlpba_md')
nlp.disable_pipes("tagger", "parser")
nlp.add_pipe("sentencizer")
linker = pyobo.get_scispacy_entity_linker("hgnc", filter_for_definitions=False, resolve_abbreviations=True,)


In [139]:
ncbi_base_url='https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
query='"signaling pathway"[Title/Abstract] AND JAK/STAT AND review[Publication Type] AND (humans[MeSH Terms] AND ("disease"[MeSH Terms] OR disease[Title/Abstract]))'
params_id = {"db": "pubmed","term": query,"retmode": "json","retstart": 0,"retmax": 10000}
response=requests.get(ncbi_base_url, params=params_id)
pubmed_id=response.json()['esearchresult']['idlist']


In [ ]:
pub_2_pmc={}
pubmed_2_PMC_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi"
for i in pubmed_id:
    params_PMC = {"dbfrom": "pubmed","db": "pmc","id": i,"retmode": "json"}
    response = requests.get(pubmed_2_PMC_url, params=params_PMC)
    id=response.json()['linksets'][0]
    if 'linksetdbs' in id.keys():
        pub_2_pmc[i]=int(id['linksetdbs'][0]['links'][0])

In [144]:
all_docs=[]
for i in pubmed_id: 
    if i in pub_2_pmc.keys():
        doc=WebBaseLoader(f'https://pmc.ncbi.nlm.nih.gov/articles/{pub_2_pmc.get(i)}/').load()
        all_docs.extend(doc)


In [179]:
all_docs[0].page_content.split('References')[:-1]

['Interstitial lung disease (ILD) is a significant extra-articular complication of rheumatoid arthritis (RA), characterized by high prevalence and mortality rates. Although advancements have been made in understanding its potential mechanisms, the pathogenesis of RA-associated ILD remains incompletely understood. Recent research has shed light on roles of various disease-related signaling pathways, including TGF-β/SMAD, JAK/STAT, PI3K–Akt, Wnt/β-catenin, and NF-κB, which are implicated in development of both RA and lung fibrosis. These shared pathways, which drive CCL1 PROTEIN production and fibroblast proliferation, offer promising opportunities for therapeutic intervention, including pathway-specific inhibition and drug repurposing. Furthermore, the growing identification of potential biomarkers for early detection and severity assessment in RA-ILD patients holds promise for improving clinical management and guiding treatment strategies. Current treatments fall short in effectively h

In [181]:
#doc=WebBaseLoader('https://pmc.ncbi.nlm.nih.gov/articles/PMC11554381/').load()
cleaned_docs=[]
count=0
for doc in all_docs:
    if 'Reference' in doc.page_content and 'Abstract' in doc.page_content:
        references=doc.page_content.split('.References')[:-1]
        title=doc.metadata['title'].strip()
        cleaning=re.sub(r"\s+", " ", doc.page_content)
        cleaning=re.split(r"Abstract",cleaning)[1]
        cleaning=re.split(r"Acknowledgements",cleaning)[0]
        cleaning=re.sub(r'.Keywords*\n?',' ',cleaning)
        cleaning=re.sub('Open in a new tab','',cleaning)
        cleaning=re.sub(r'[\r\n]+', r'\n', cleaning)
        cleaning=re.sub(r"\s*\(?\s*(?:Figure|Fig\.?)\s*[\d.]+\s*\)?\s*",'',cleaning,flags=re.IGNORECASE).strip()
        cleaning=re.sub(r'\[[\d\s\W]*?\].','',cleaning,flags=re.IGNORECASE)
        cleaning=re.sub(r"\(([^)]*?\s*et\s*al\.\s*[^)]*?)\)",'',cleaning)
        cleaning=re.sub(r'\(\d+(?:,\d+|-?\d+)*\)\.','.',cleaning)
        cleaning=re.sub(r'\([\w\s]+\d{4},?\s*[\d–-]*\)','',cleaning)
        cleaning=re.sub(r"\([rR]eviewed in\)",'',cleaning,flags=re.IGNORECASE)
        cleaning=re.sub(r"www\.[^/]+/",' ',cleaning)
        cleaned_nlp=linker(nlp(cleaning))
        new_text=cleaning
        for i in reversed(cleaned_nlp.ents):
            id=i._.kb_ents
            if id:
                name=linker.kb.cui_to_entity[id[0][0]].canonical_name
                new_text = new_text[:i.start_char] + name + ' ' + i.label_ + new_text[i.end_char:]

        doc.page_content=new_text
        cleaned_docs.append(doc)

In [ ]:
cleaned_nlp=linker(nlp(cleaning))

In [ ]:
new_text=cleaning
for i in reversed(cleaned_nlp.ents):
    id=i._.kb_ents
    if id:
        name=linker.kb.cui_to_entity[id[0][0]].canonical_name
        new_text = new_text[:i.start_char] + name + ' ' + i.label_ + new_text[i.end_char:]

doc[0].page_content=new_text

In [ ]:
with open('/home/bionik/AI_ML/FDA_Project/LLM/doc_list.pkl', 'wb') as f: # Open in write binary ('wb') mode
    pickle.dump(cleaned_docs, f)

In [ ]:
with open('/home/bionik/AI_ML/FDA_Project/LLM/doc_list.pkl', 'rb') as file:
    cleaned_docs = pickle.load(file)

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=500)
chunks=splitter.split_documents(cleaned_docs)

In [24]:
embeddings_model = OllamaEmbeddings(model="qwen3-embedding:0.6b",validate_model_on_init=True)
vectorstore=FAISS.from_documents(chunks[:2],embeddings_model)


In [25]:
list_of_chunks = [chunks[2:][i:i + 5] for i in range(0, len(chunks[2:]), 5)]

In [27]:
count=0
for i in list_of_chunks:
    try:
        print(count)
        count+=1
        vectorstore.add_documents(i)
    except Exception as e:
        print(e)
        continue

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [29]:
retriever=vectorstore.as_retriever(search_type='similarity',search_kwargs={"k": 20, "fetch_k": 20})

In [31]:
vectorstore.save_local("/home/bionik/AI_ML/FDA_Project/LLM/faiss_index")

In [44]:
class Triplet(BaseModel):
    subject: str = Field(..., description="ONE protein or ONE protein complex")
    predicate: str = Field(..., description="Relationship between protein entities")
    object: str = Field(..., description="ONE protein or ONE protein complex")

class TripletList(BaseModel):
    triplets: List[Triplet]

In [45]:
template = """
You are an expert information extraction assistant. 
Your task is to extract knowledge triples in the format of (subject, predicate, object) from the provided text. A subject and object are protein entities, and the predicate describes the relationship between them.

**Instructions:**
1.  Identify all relevant subject-predicate-object triples that represent facts stated explicitly in the text.
2.  Focus on extracting facts accurately and completely.
3.  Do not hallucinate or add information that is not present in the source text.
4.  If a sentence contains multiple facts, extract each fact as a separate triple.
5.  Predicate values should be only from ['Activates','Inhibits','Reacts','Form Complex']
6.  Subject and object should be only proteins.

Normalize synonyms:
binds/interacts/phosphorylate → Reacts
forms a complex → Form Complex
suppresses/destabilizes/activates degradation → Inhibits
induces/promotes/activates/stabilizes/inhibits degradation → Activates

Retrieve triplets from this text below:
{context}

"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
llm=ChatOllama(model='qwen3:4b-instruct-2507-q4_K_M',temperature=0.1,validate_model_on_init=True,num_ctx=12000) ## this is better
#llm=ChatOllama(model='qwen3:4b-instruct-2507-q4_K_M',temperature=0.0,validate_model_on_init=True,num_ctx=12000).with_structured_output(TripletList) ## this is better



In [48]:
rag_chain = (RunnableParallel(context=retriever,question=RunnablePassthrough())
    | prompt
    | llm)

In [49]:
response = rag_chain.invoke("What are the CDC37 proteins's key characteristics/interactions?")

In [33]:
system_template = """
Use the following pieces of retrieved context to answer the question of research article of biology. 
If you don't know the answer, genuinely say that you don't know.
Use only the information which was provided in this query
Keep the answer detailed and precise.

The question is:
{question}

Use this context to answer the question:
{context}
"""

prompt = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template(system_template),
                                           HumanMessagePromptTemplate.from_template('{question}')])

In [36]:
rag_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
)

In [ ]:
ans=llm.invoke('explain details EMT and which pathways are involved in this process?')

KeyboardInterrupt: 

In [ ]:
ans

In [ ]:
input_data = 'explain details EMT and which pathways are involved in this process?'
response = rag_chain.invoke(input_data)

In [ ]:
print(response.content)

{'input_tokens': 21, 'output_tokens': 1442, 'total_tokens': 1463}

In [40]:
retriever.invoke('explain details EMT and which pathways are involved in this process?')

[Document(id='977c2c03-2d89-49a0-b10b-ddf2fda5c124', metadata={'source': 'https://pmc.ncbi.nlm.nih.gov/articles/9822656/', 'title': '\n            Acute Myeloid Leukemia Following Myeloproliferative Neoplasms: A Review of What We Know, What We Do Not Know, and Emerging Treatment Strategies - PMC\n        ', 'description': 'Acute myeloid leukemia (AML) arising from myeloproliferative neoplasms (MPNs) represents a small subtype of secondary AML (sAML). This entity is well known to be associated with poor responses to available treatment options and dismal outcomes. To ...', 'language': 'en'}, page_content='collection Name your collection * Choose a collection Unable to load your collection due to an error Please try again Add Cancel Follow NCBI NCBI on X (formerly known as Twitter) NCBI on Facebook NCBI on LinkedIn NCBI on GitHub NCBI RSS feed Connect with NLM NLM on X (formerly known as Twitter) NLM on Facebook NLM on YouTube National Library of Medicine 8600 Rockville Pike Bethesda, MD

In [ ]:
engine = pyttsx3.init()
output='Usage'
text = f"The output of Large Language model: {response}"
engine.setProperty('rate', 160)
engine.setProperty('volume',0.7)
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[23].id)
engine.say(text)
engine.runAndWait()

aplay: pcm_write:2146: write error: Interrupted system call


In [73]:
docs_content = "\n\n".join(doc.page_content for doc in vectorstore.similarity_search('which Signalling pathways are involved in human diseases'))

In [74]:
docs_content

'The GHR PROTEIN signaling pathway plays important roles in growth, metabolism, cell cycle control, immunity, homeostatic processes, and chemoresistance via both the JAK/STAT and the SRC PROTEIN pathways. Dysregulation of GHR PROTEIN signaling is associated with various diseases and chronic conditions such as acromegaly, cancer, aging, metabolic disease, fibroses, inflammation and autoimmunity. Numerous studies entailing the GHR PROTEIN signaling pathway have been conducted for various cancers. Diverse factors mediate the up- or down-regulation of GHR PROTEIN signaling through post-translational modifications. Of the numerous modifications, ubiquitination and deubiquitination are prominent events. Ubiquitination by PRKN PROTEIN attaches UBD PROTEIN to target proteins and induces proteasomal degradation or starts the sequence of events that leads to endocytosis and lysosomal degradation. In this review, we discuss the role of first line effectors that act directly on the GHR PROTEIN at 